In [1]:
import numpy as np
from matplotlib import pyplot as plt
import random
import warnings


warnings.filterwarnings('ignore')  # 抑制警告显示

sample = np.load('../numpy/sample_process.npy')
sample_time = np.load('../numpy/sample_time.npy')

cycle = 567

plt.rcParams['savefig.dpi'] = 200  # 图片像素
plt.rcParams['figure.dpi'] = 200  # 分辨率
plt.rcParams['font.sans-serif']=['SimHei']  # 正常显示中文标签
plt.rcParams['axes.unicode_minus']=False  # 正常显示负号

plt.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=None, hspace=None)

<Figure size 1200x800 with 0 Axes>

## 三个方法
---


* `append_np`: 将多个一维`numpy.array`合并为一个


* `add_mutation_fault`: 添加突变故障(维度，左区间端点，右区间端点，偏差)


* `add_gradient_fault`: 添加渐变故障(维度，左区间端点，右区间端点，偏差，该维度数据的下限，渐变次数)

In [2]:
def append_np(*args):
    np_new = args[0]
    for item in args[1:]:
        np_new = np.append(np_new, item)
    return np_new


def add_mutation_fault(sample_np, a, b, bias):
    sample_fault = np.array([])
    for item in sample_np[a:b]:
        temp = item -bias
        if temp <= 0:
            sample_fault = np.append(sample_fault, 0)
        else:
            sample_fault = np.append(sample_fault, temp)
    
    return append_np(sample_np[:a], sample_fault, sample_np[b:])


def add_gradient_fault(sample_np, a, b, e, data_min):
    x = np.array(range(0, b - a))
    y = np.power(e, -x)
    sample_fault = sample_np[a:b] * y + data_min * (1 - y)

    return append_np(sample_np[:a], sample_fault, sample_np[b:])


## 正常数据(normal)
---

In [3]:
normal_ort = np.array([])  # 储存按时间排序的current数据
sample_normal = np.array([])  # 储存按电压排序的current数据

for i in range(int(sample.shape[1] / cycle)):
    
    start = i * cycle  # 本周期段起点
    end = (i + 1) * cycle  # 本周期段终点
    
    # 按照时间排序current数据
    normal_ort = np.append(normal_ort, sample[0, start:end])
    
    # 将current数据按照voltage大小排序，并添加到sample_normal
    index = np.argsort(sample[1, start:end])
    temp = np.array([sample[0, start:end][item] for item in index])
    sample_normal = np.append(sample_normal, temp)
    
    # 绘图s-A-V并保存
    ax = plt.gca(projection='3d')
    ax.set(xlabel='time(s)', ylabel='current(A)', zlabel='voltage(V)')
    ax.scatter(sample_time[start:end], sample[0, start:end], sample[1, start:end], marker='+', s=1)
    plt.savefig('../pic/s-A-V/normal/%d-%d.png' % (i, i+1), transparent=True)
    plt.clf()  # 清空画板
    
np.save('../numpy/V-A/normal.npy', sample_normal)


<Figure size 1200x800 with 0 Axes>

## 电路故障(fault_1)
---

In [4]:
fault_1_ort = np.array([])  # 储存按时间排序的current数据
sample_fault = np.array([])  # 储存按电压排序的current数据

for i in range(int(sample.shape[1] / cycle)):
    
    start = i * cycle  # 本周期段起点
    end = (i + 1) * cycle  # 本周期段终点
    
    # 生成故障数据(f_s)
    f_start = random.randint(0, int(cycle/3))
    f_end = random.randint(int(cycle/2), cycle)
    f_s = add_mutation_fault(sample[0, start:end], f_start, f_end, random.uniform(3, 5))
    
    # 按照时间排序current故障数据
    fault_1_ort = np.append(fault_1_ort, f_s)
    
    # 将current数据按照voltage大小排序，并添加到sample_fault
    index = np.argsort(sample[1, start:end])
    temp = np.array([f_s[item] for item in index])
    sample_fault = np.append(sample_fault, temp)
    
    # 绘图s-A-V并保存
    ax = plt.gca(projection='3d')
    ax.set(xlabel='time(s)', ylabel='current(A)', zlabel='voltage(V)')
    ax.scatter(sample_time[start:end], f_s, sample[1, start:end], marker='+', s=1)
    plt.savefig('../pic/s-A-V/fault_1/%d-%d.png' % (i, i+1), transparent=True)
    plt.clf()  # 清空画板
    
np.save('../numpy/V-A/fault_1.npy', sample_fault)


<Figure size 1200x800 with 0 Axes>

## 展开机构故障(fault_2)
---

In [5]:
fault_2_ort = np.array([])  # 储存按时间排序的current数据
sample_fault = np.array([])  # 储存按电压排序的current数据

for i in range(int(sample.shape[1] / cycle)):
    
    start = i * cycle  # 本周期段起点
    end = (i + 1) * cycle  # 本周期段终点
    
    # 生成故障数据(f_s)
    f_start = random.randint(0, int(cycle/3))
    f_end = random.randint(int(cycle/2), cycle)
    f_s = add_mutation_fault(sample[0, start:end], f_start, f_end, random.uniform(8, 10))
    
    # 按照时间排序current故障数据
    fault_2_ort = np.append(fault_2_ort, f_s)
    
    # 将current数据按照voltage大小排序，并添加到sample_fault
    index = np.argsort(sample[1, start:end])
    temp = np.array([f_s[item] for item in index])
    sample_fault = np.append(sample_fault, temp)
    
    # 绘图s-A-V并保存
    ax = plt.gca(projection='3d')
    ax.set(xlabel='time(s)', ylabel='current(A)', zlabel='voltage(V)')
    ax.scatter(sample_time[start:end], f_s, sample[1, start:end], marker='+', s=1)
    plt.savefig('../pic/s-A-V/fault_2/%d-%d.png' % (i, i+1), transparent=True)
    plt.clf()  # 清空画板
    
np.save('../numpy/V-A/fault_2.npy', sample_fault)


<Figure size 1200x800 with 0 Axes>

## 对日定向驱动故障(fault_3)
---

In [6]:
fault_3_ort = np.array([])  # 储存按时间排序的current数据
sample_fault = np.array([])  # 储存按电压排序的current数据

for i in range(int(sample.shape[1] / cycle)):
    
    start = i * cycle  # 本周期段起点
    end = (i + 1) * cycle  # 本周期段终点
    
    # 生成故障数据(f_s)
    f_start = random.randint(int(cycle/8), int(cycle/2.5))
    f_end = cycle
    f_s = add_gradient_fault(sample[0, start:end], f_start, f_end, random.uniform(1.001, 1.005), 0)
    
    # 按照时间排序current故障数据
    fault_3_ort = np.append(fault_3_ort, f_s)
    
    # 将current数据按照voltage大小排序，并添加到sample_fault
    index = np.argsort(sample[1, start:end])
    temp = np.array([f_s[item] for item in index])
    sample_fault = np.append(sample_fault, temp)
    
    # 绘图s-A-V并保存
    ax = plt.gca(projection='3d')
    ax.set(xlabel='time(s)', ylabel='current(A)', zlabel='voltage(V)')
    ax.scatter(sample_time[start:end], f_s, sample[1, start:end], marker='+', s=1)
    plt.savefig('../pic/s-A-V/fault_3/%d-%d.png' % (i, i+1), transparent=True)
    plt.clf()  # 清空画板
    
np.save('../numpy/V-A/fault_3.npy', sample_fault)


<Figure size 1200x800 with 0 Axes>

## 将按时间顺序排序的故障样本保存
---

In [7]:
sample_fault = np.array([normal_ort, fault_1_ort, fault_2_ort, fault_3_ort])
np.save('../numpy/sample_fault.npy', sample_fault)